In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

22/09/29 14:29:07 WARN Utils: Your hostname, EKTAs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.100.4 instead (on interface en0)
22/09/29 14:29:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/29 14:29:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/29 14:29:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/29 14:29:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [15]:
df=spark.read.csv('Cricket.csv',header=True, inferSchema=False)
spark.read.csv('Cricket.csv',header=True, inferSchema=True).show()

+--------+----+----------+------+
|    Name| Age|Experience|Salary|
+--------+----+----------+------+
|   Virat|  34|        10| 60000|
|   Rohit|  35|        13| 54000|
|  Pandya|  29|         5| 30000|
|   Panth|  25|         2| 20000|
|   Dhoni|  40|        19| 51000|
|   Rahul|  31|         6| 23000|
| Shubham|null|      null| 17000|
|Ravindra|  23|        10| 87000|
|    null|  39|      null|  null|
+--------+----+----------+------+



## Drop the column

In [3]:
df.drop('Name').show()

+----+----------+------+
| Age|Experience|Salary|
+----+----------+------+
|  34|        10| 60000|
|  35|        13| 54000|
|  29|         5| 30000|
|  25|         2| 20000|
|  40|        19| 51000|
|  31|         6| 23000|
|null|      null| 17000|
|  23|        10| 87000|
|  39|      null|  null|
+----+----------+------+



# Handling null values

In [4]:
df.na.drop().show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|   Virat| 34|        10| 60000|
|   Rohit| 35|        13| 54000|
|  Pandya| 29|         5| 30000|
|   Panth| 25|         2| 20000|
|   Dhoni| 40|        19| 51000|
|   Rahul| 31|         6| 23000|
|Ravindra| 23|        10| 87000|
+--------+---+----------+------+



df.na.drop(how='any').show() # if any value in a row has null values (default)
df.na.drop(how='all').show() # if all values in a row is null

In [5]:
# threshold -> minimum these many values needs to be null in a row

df.na.drop(how='any',thresh=2).show()

+--------+----+----------+------+
|    Name| Age|Experience|Salary|
+--------+----+----------+------+
|   Virat|  34|        10| 60000|
|   Rohit|  35|        13| 54000|
|  Pandya|  29|         5| 30000|
|   Panth|  25|         2| 20000|
|   Dhoni|  40|        19| 51000|
|   Rahul|  31|         6| 23000|
| Shubham|null|      null| 17000|
|Ravindra|  23|        10| 87000|
+--------+----+----------+------+



In [6]:
# subset = remove null value only from a specific column

df.na.drop(how='any', subset=['Experience']).show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|   Virat| 34|        10| 60000|
|   Rohit| 35|        13| 54000|
|  Pandya| 29|         5| 30000|
|   Panth| 25|         2| 20000|
|   Dhoni| 40|        19| 51000|
|   Rahul| 31|         6| 23000|
|Ravindra| 23|        10| 87000|
+--------+---+----------+------+



## Filling missing values

In [16]:
df.fillna('Missing value').show()

+-------------+-------------+-------------+-------------+
|         Name|          Age|   Experience|       Salary|
+-------------+-------------+-------------+-------------+
|        Virat|           34|           10|        60000|
|        Rohit|           35|           13|        54000|
|       Pandya|           29|            5|        30000|
|        Panth|           25|            2|        20000|
|        Dhoni|           40|           19|        51000|
|        Rahul|           31|            6|        23000|
|      Shubham|Missing value|Missing value|        17000|
|     Ravindra|           23|           10|        87000|
|Missing value|           39|Missing value|Missing value|
+-------------+-------------+-------------+-------------+



In [17]:
df.fillna('Missing value','Experience').show()

+--------+----+-------------+------+
|    Name| Age|   Experience|Salary|
+--------+----+-------------+------+
|   Virat|  34|           10| 60000|
|   Rohit|  35|           13| 54000|
|  Pandya|  29|            5| 30000|
|   Panth|  25|            2| 20000|
|   Dhoni|  40|           19| 51000|
|   Rahul|  31|            6| 23000|
| Shubham|null|Missing value| 17000|
|Ravindra|  23|           10| 87000|
|    null|  39|Missing value|  null|
+--------+----+-------------+------+



In [18]:
df.fillna('Missing value',['Experience','Age']).show()

+--------+-------------+-------------+------+
|    Name|          Age|   Experience|Salary|
+--------+-------------+-------------+------+
|   Virat|           34|           10| 60000|
|   Rohit|           35|           13| 54000|
|  Pandya|           29|            5| 30000|
|   Panth|           25|            2| 20000|
|   Dhoni|           40|           19| 51000|
|   Rahul|           31|            6| 23000|
| Shubham|Missing value|Missing value| 17000|
|Ravindra|           23|           10| 87000|
|    null|           39|Missing value|  null|
+--------+-------------+-------------+------+



### To fill the missing values with either mean, median or mode, we will use Imputer function

In [19]:
from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols=['Age','Experience','Salary'],
                 outputCols=['{}_imputed'.format(c) for c in ['Age','Experience','Salary']]).setStrategy('mean')

In [21]:
# Add imputation cols to df

df=spark.read.csv('Cricket.csv',header=True, inferSchema=True)

imputer.fit(df).transform(df).show()

+--------+----+----------+------+-----------+------------------+--------------+
|    Name| Age|Experience|Salary|Age_imputed|Experience_imputed|Salary_imputed|
+--------+----+----------+------+-----------+------------------+--------------+
|   Virat|  34|        10| 60000|         34|                10|         60000|
|   Rohit|  35|        13| 54000|         35|                13|         54000|
|  Pandya|  29|         5| 30000|         29|                 5|         30000|
|   Panth|  25|         2| 20000|         25|                 2|         20000|
|   Dhoni|  40|        19| 51000|         40|                19|         51000|
|   Rahul|  31|         6| 23000|         31|                 6|         23000|
| Shubham|null|      null| 17000|         32|                 9|         17000|
|Ravindra|  23|        10| 87000|         23|                10|         87000|
|    null|  39|      null|  null|         39|                 9|         42750|
+--------+----+----------+------+-------